In [44]:
# import libraries
import pandas as pd
import numpy as np
from functools import reduce
#create my final df for all my data
final_df = pd.DataFrame()
#ignore this
excluders = [6, 8, 10, 13, 15, 17, 20, 24, 26, 31, 34, 37]
# keeping track of how many matches have been process, ignore for now
matchcount = 0
# If you are uploading 25 matches from your desktop, games should = 26, alter accordingly
games = 51
# IMPORTANT – the teamid field must match the id of the team you want the data for in the file 
teamid = 32
# read in PV data and alter it for our purposes
pv = pd.read_csv("Desktop/PV.csv", names = ["coord", "val"])
x=pv["coord"].str.split(',', expand=True)
pv = pd.concat([pv, x], axis=1)
pv.drop(columns = ["coord"], inplace=True)
pv.columns = ['pv', 'x', 'y']
pv['x'] = pv['x'].str[1:]
pv['y'] = pv['y'].str[:-1]
pv['x'] = pv['x'].astype(float)
pv['y'] = pv['y'].astype(float)
for i in range(1, games):
    if i not in excluders:
        #increase how many matches we have processed in the counter
        matchcount +=1
        # everyone file you upload should be on your desktop in the format "match1.csv", "match2.csv", etc etc
        string = "Desktop/match" + str(i) + ".csv"
        # read in the file
        df = pd.read_csv(string)
        # limit the dataframe to only the following fields
        df = df[["id", "eventId", "x", "y","playerId", "teamId", "relatedPlayerId", "type/displayName", "outcomeType/displayName","endX", "endY"]]
        # limit the event types in the dataframe to the following values
        actiontypes1= ['Pass', 'Tackle', 'TakeOn', 'Dispossessed', 'BallRecovery',
         'SavedShot', 'BallTouch', 'Interception', 'Challenge',
         'CornerAwarded', 'MissedShots','Foul', 'Clearance',
         'BlockedPass', 'OffsideGiven', 'OffsidePass', 'ShieldBallOpp', 'Goal', 'Claim',
         'ShotOnPost', 'GoodSkill']
        df = df.loc[df['type/displayName'].isin(actiontypes1)]
        # further limitation of event types for later
        actiontypes2 = ['Pass', 'BallTouch',
       'Dispossessed', 'Clearance', 'SavedShot', 'TakeOn',
       'MissedShots', 'CornerAwarded', 'BlockedPass', 'ShotOnPost', 'Goal', 'ShieldBallOpp', 'OffsideGiven', 'OffsidePass', 'GoodSkill', 'ChanceMissed']
        # scale data to our scale
        df['x']=df['x'] * 0.5
        df['y']=df['y'] * 0.32 
        df["endX"] = df['endX'] * 0.49
        df["endY"]=df['endY'] * 0.31
        df['x']=df['x'].apply(np.floor)
        df['y']=df['y'].apply(np.floor) 
        df["endX"] = df['endX'].apply(np.floor)
        df["endY"]=df['endY'].apply(np.floor)
        # merge the xy data with the pv data
        pv.columns = ['pv', 'x', 'y']
        dfs = [df, pv]
        df = reduce(lambda left,right: pd.merge(left,right, how='left', on=['x','y']), dfs)
        pv.columns = ["endpv", "endX", "endY"]
        dfs = [df, pv]
        df = reduce(lambda left,right: pd.merge(left,right, how='left', on=['endX','endY']), dfs)
        # limit events again for later
        validevents = ["BallTouch", "TakeOn", "ShieldBallOpp", "GoodSkill", "Tackle", "BallRecovery", "Interception"]
        # create a list of rows that we will iterate through (this is typically bad practice but there's a reason)
        mark = df["teamId"] == teamid
        # set default values for new fields
        df["received_pv"] = 0
        df["received_x"] = 0
        df["received_y"] = 0
        df["carried_pv"] = 0
        df["carried_start_x"] = 0
        df["carried_start_y"] = 0
        # use masks to limit the negative attribution of PV based on ball losses
        mask = (df['outcomeType/displayName'] == "Unsuccessful") & (df['pv'] > 0.025)
        df['endpv'][mask] = df['pv'] - 0.025
        mask2 = (df['outcomeType/displayName'] == "Unsuccessful") & (df['pv'] <= 0.025)
        df['endpv'][mask2] = 0
        # create net_pv field
        df["net_pv"] = df["endpv"] - df["pv"]
        # iterate through team events and attribute values
        for p in range(len(mark)):
            # if the row is a team event
            if mark[p] == True:
                # and the row is not the last row
                if p + 1 < len(df["id"]):
                    # then select the row
                    row = df.iloc[ p, : ]
                    # attributing reception and carry value on passes
                    if (row["type/displayName"] == 'Pass' and row['outcomeType/displayName'] == "Successful"):
                        df.loc[p+1,'received_pv'] = df.at[p,"net_pv"]
                        df.loc[p+1,'received_x'] = df.at[p,"endX"]
                        df.loc[p+1,'received_y'] = df.at[p,"endY"]
                        df.loc[p+1, 'carried_pv'] = df.at[p+1, 'pv'] - df.at[p, 'endpv']
                        df.loc[p+1,'carried_start_x'] = df.at[p,"endX"]
                        df.loc[p+1,'carried_start_y'] = df.at[p,"endY"]
                    # atrributing carry values otherwise
                    elif (row["type/displayName"] in validevents and row["playerId"] == df.at[p+1, "playerId"]):
                        df.loc[p+1, 'carried_pv'] = df.at[p+1, 'pv'] - df.at[p, 'endpv']
                        df.loc[p+1,'carried_start_x'] = df.at[p,"endX"]
                        df.loc[p+1,'carried_start_y'] = df.at[p,"endY"]
        # limiting action types to only those that may receive value attribution
        df = df.loc[df['type/displayName'].isin(actiontypes2)]
        # limiting rows to team rows
        df = df[df["teamId"] == teamid]
        # add current df to final df
        final_df = pd.concat([final_df, df], axis=0)
print(matchcount)

/Users/keesvanhemmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/keesvanhemmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


38


In [18]:
# removes box entries (only run if you want Staging PV)
ends = final_df[final_df["endX"] < 42]
endh = final_df[final_df["endY"] < 4]
endl = final_df[final_df["endY"] > 28]


final_df = pd.concat([ends, endh, endl], axis=0)

final_df.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = True) 

In [26]:
final_df

,id,eventId,x,y,playerId,teamId,relatedPlayerId,type/displayName,outcomeType/displayName,endX,endY,pv,endpv,received_pv,received_x,received_y,carried_pv,carried_start_x,carried_start_y,net_pv


In [ ]:
# allow only box entries (only run if you want to isolate box entries)
final_df = final_df[final_df["endY"] > 4]
final_df = final_df[final_df["endy"] < 28]

In [ ]:
# remove final third entries (only run if you want Buildup PV)
final_df = final_df[final_df["x"] > 34]
final_df = final_df[final_df["endX"] < 34]

In [29]:
# read in the minutes file from the desktop where you map playerId to playerName and minutes
# make sure you file is named 'min.csv', or change the filename in the code
# this can just be an excel file that you save as a csv
minutes = pd.read_csv('Desktop/minutes.csv')

In [46]:
# sets the field names – make sure your fields have the same names, case sensitive
minutes = minutes[["playerId", "Name", "Minutes"]]

In [45]:
print(final_df["playerId"].unique())

[122366. 109000. 336915. 118244. 300299. 331924.  79554.  99487.  97752.
 243254. 100008. 302313. 367782.  25363.   8166.  38128. 114397. 365244.
 299271. 338497. 353418.  70050. 367781. 384143.  81726. 123761. 243814.
  31376. 312739.]


In [47]:
# merge minutes
dfs = [final_df, minutes]
df = reduce(lambda left,right: pd.merge(left,right, on='playerId'), dfs)

In [48]:
# write the data to a csv on your desktop (you can open in tableau with the Text File option)
df.to_csv("Desktop/unitedpvpv2.csv")